In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Random dataset

In [ ]:
def random_bool(size):
    return np.random.rand(size) > 0.5

def random_int(min, max, size):
    return np.random.randint(min, max, size=size)

def random_float(min, max, size):
    return np.random.uniform(min, max, size=size)

def index_id(max):
    list = []
    for i in range(1, max):
        list.append('ID' + str(i))
    return list

In [ ]:
pre_op = pd.DataFrame({'IDno': index_id(501), 'Død inden for 1 år af operation': random_bool(500), 'age': random_int(18, 89, 500), 'sex': random_bool(500), 'Respiratory Disease': random_bool(500),
       'Circulatory Organ Disease': random_bool(500), 'Type 1 Diabetes': random_bool(500), 'Type 2 Diabetes': random_bool(500),
       'Other metabolic diseases': random_bool(500), 'Other operation': random_bool(500), 'Genital or urine related diseases': random_bool(500), 'Vægt': random_float(35, 190,500), 'KFN': random_bool(500), 'KFX': random_bool(500), 'KFK': random_bool(500), 'KFM': random_bool(500),
       'KFF': random_bool(500), 'KFP': random_bool(500), 'KFC': random_bool(500), 'KFW': random_bool(500), 'KFJ': random_bool(500), 'Hæmoglobin': random_float(4,12,500), 'Leukocytter': random_float(1, 125, 500),
       'Trombocytter': random_float(20, 1000, 500)})
pre_op = pre_op.replace(True, 1).replace(False, 0)

# Predicting Mortality with pre_op data

## Logistic regression LR

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

#TODO other params?
def lr_pred(X_train, y_train, X_test):
    lr = LogisticRegression()
    lr_grid = {'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
            'C': [100, 10, 1.0, 0.1, 0.01],
                'max_iter': [200,300]}
    
    grid_result = GridSearchCV(estimator = lr, param_grid = lr_grid, scoring='precision', cv=5)
    grid_result.fit(X_train,y_train)
    
    best_model = grid_result.best_estimator_ 

    lr_preds = best_model.predict_proba(X_test)[:,1]
    return lr_preds

    #TODO Take Best result of the fit or can i just return the predict 

## Random forest (RF)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

def rf_pred(X_train, y_train, X_test):
    rf = RandomForestClassifier()

    rf_grid = {'max_depth': [80, 90, 100, 110],
    'max_features': [2,5,10,15,22],
    'n_estimators': [100, 200, 300, 1000]}
    
    grid_result = GridSearchCV(estimator = rf, param_grid = rf_grid, scoring='precision', cv=5)
    grid_result.fit(X_train,y_train)
    
    best_model = grid_result.best_estimator_ 

    rf_preds = best_model.predict_proba(X_test)[:,1]
    return rf_preds

## Gradient boost

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Define the target and features
target = 'Død inden for 1 år af operation'
features = [col for col in pre_op.columns if col != target and col != 'IDno']

X = pre_op[features]
y = pre_op[target]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

class GradientBoosting:
    def __init__(self, balanced=False):
        self.balanced = balanced
        self.model = None

    def create_model(self, random_state):
        params = {
            "max_depth": 2,
            "learning_rate": 1,
            "n_estimators": 10,
            "subsample": 0.5,
            "random_state": random_state.get_state()[1][0],
        }
        if self.balanced:
            params["scale_pos_weight"] = "balanced"
        self.model = GradientBoostingClassifier(**params)
        return self.model

    def fit_model(self, model, train_data, train_labels):
        model.fit(train_data, train_labels)

    def predict_model(self, model, test_data):
        preds = model.predict_proba(test_data)[:, 1]
        return preds

    @staticmethod
    def save_model(model, path):
        import joblib
        joblib.dump(model, path)

# Initialize the model
gb_model = GradientBoosting()

# Create the model parameters
model = gb_model.create_model(np.random.RandomState(42))

# Train the model
gb_model.fit_model(model, X_train, y_train)

# Make predictions on the test set
predictions = gb_model.predict_model(model, X_test)

# Convert probabilities to binary outcomes
threshold = 0.5
binary_predictions = (predictions >= threshold).astype(int)

# Convert the probabilities to a numpy array and print it
probabilities_array = np.array(predictions)
print(probabilities_array)

# Evaluate the binary predictions
accuracy = accuracy_score(y_test, binary_predictions)
conf_matrix = confusion_matrix(y_test, binary_predictions)
class_report = classification_report(y_test, binary_predictions)

In [ ]:
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

## MLP

# Grouped cross-validation

In [ ]:
from sklearn.metrics import precision_recall_curve, roc_curve, auc, average_precision_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GroupKFold

X = pre_op.loc[:, pre_op.columns != 'Død inden for 1 år af operation']
X = X.loc[:, X.columns != 'IDno']
y = pre_op['Død inden for 1 år af operation']

outer_cv = GroupKFold(n_splits = 5)
inner_cv = GroupKFold(n_splits = 5)

#TODO add gbm and mlp into this dictionary
results = {
    'lr': {'pr_auc': [], 'roc_auc': [], 'precision': [], 'recall': [], 'fpr': [], 'tpr': []},
    'rf': {'pr_auc': [], 'roc_auc': [], 'precision': [], 'recall': [], 'fpr': [], 'tpr': []}
}

def evaluate_model(pred_func, model_name):
    for outer_train_idx, outer_test_idx in outer_cv.split(X, y, groups = pre_op['IDno']):
        X_outer_train, y_outer_train = X.iloc[outer_train_idx], y.iloc[outer_train_idx]
        X_outer_test, y_outer_test = X.iloc[outer_test_idx], y.iloc[outer_test_idx]

        inner_cv = GroupKFold(n_splits=5)
        inner_pr_auc_scores = []
        inner_roc_auc_scores = []

        for inner_train_idx, inner_test_idx in inner_cv.split(X_outer_train, y_outer_train, groups=pre_op['IDno'].iloc[outer_train_idx]):
            X_inner_train, y_inner_train = X_outer_train.iloc[inner_train_idx], y_outer_train.iloc[inner_train_idx]
            X_inner_test, y_inner_test = X_outer_train.iloc[inner_test_idx], y_outer_train.iloc[inner_test_idx]
            #TODO Make the perfomance class that returns PR AUC, ROC AUC and PR and ROC

            y_pred_prob = pred_func(X_inner_train, y_inner_train, X_inner_test)
    
            precision, recall, _ = precision_recall_curve(y_inner_test, y_pred_prob)
            pr_auc = auc(recall, precision)
            inner_pr_auc_scores.append(pr_auc)

            fpr, tpr, _ = roc_curve(y_inner_test, y_pred_prob)
            roc_auc = roc_auc_score(y_inner_test, y_pred_prob)
            inner_roc_auc_scores.append(roc_auc)

        y_pred_prob = pred_func(X_outer_train, y_outer_train, X_outer_test)

        # Compute PR AUC
        precision, recall, _ = precision_recall_curve(y_outer_test, y_pred_prob)
        pr_auc = auc(recall, precision)
        results[model_name]['pr_auc'].append(pr_auc)

        # Compute ROC AUC
        fpr, tpr, _ = roc_curve(y_outer_test, y_pred_prob)
        roc_auc = roc_auc_score(y_outer_test, y_pred_prob)
        results[model_name]['roc_auc'].append(roc_auc)

        results[model_name]['precision'].append(precision)
        results[model_name]['recall'].append(recall)
        results[model_name]['fpr'].append(fpr)
        results[model_name]['tpr'].append(tpr)

# ROC AUC and PR AUC calculations

In [ ]:
evaluate_model(lr_pred, 'lr')

In [ ]:
evaluate_model(rf_pred, 'rf')

In [ ]:
def plot_results(model_name):
    plt.figure(figsize=(14, 6))

    # Plot PR curves
    plt.subplot(1, 2, 1)
    for i in range(len(results[model_name]['precision'])):
        plt.plot(results[model_name]['recall'][i], results[model_name]['precision'][i], alpha=1, label=f'Outer fold {i+1}', linewidth=2)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'PR curve for {model_name.upper()}')
    plt.legend()

    # Plot ROC curves
    plt.subplot(1, 2, 2)
    for i in range(len(results[model_name]['fpr'])):
        plt.plot(results[model_name]['fpr'][i], results[model_name]['tpr'][i], alpha=1, label=f'Outer fold {i+1}', linewidth=2)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC curve for {model_name.upper()}')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
plot_results('lr')

In [ ]:
def print_auc_values(model_name):
    print(f"Model: {model_name.upper()}")
    for i in range(len(results[model_name]['roc_auc'])):
        print(f"Fold {i+1} - ROC AUC: {results[model_name]['roc_auc'][i]:.4f}, PR AUC: {results[model_name]['pr_auc'][i]:.4f}")
    print(f"Average ROC AUC: {np.mean(results[model_name]['roc_auc']):.4f}")
    print(f"Average PR AUC: {np.mean(results[model_name]['pr_auc']):.4f}")
    print()

In [ ]:
print_auc_values('lr')